# Procesamiento de datasets
En este notebook están agrupados los procedimientos por los cuales se importaron, concatenaron y procesaron los sets de datos originalmente provistos:
- `dataset_es_dev.json`
- `dataset_es_train.json`
- `dataset_es_test.json`

Para generar sets de datos con los atributos relevantes y las reviews lemmatizadas y stemmatizadas, segpun el caso:
- `dataset_amazon_reviews_lemma.json`
- `dataset_amazon_reviews_stem.json`

De esta forma, se ahorra tiempo en la ejecución del notebook que resulve la consigna pedida.

### Importar las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np

# Garbage collector para optimizar recursos
import gc

In [ ]:
# Previamente
## python -m spacy download es
## python -m spacy download es_core_news_sm

import spacy # https://spacy.io/usage/models
nlp = spacy.load('es_core_news_sm')

#Stop Words de es_core_news_sm
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)

# Quitamos de las stopwords palabras como 'no', 'bueno', 'buena' que nos pueden ser últiles para predecir
stopwords_spacy.remove('no') # En los comentarios negativos es muy repetida
stopwords_spacy.remove('bueno')
stopwords_spacy.remove('buena')
stopwords_spacy.remove('tarde')
stopwords_spacy.remove('temprano')
stopwords_spacy.remove('día')
stopwords_spacy.remove('días')
stopwords_spacy.remove('dia')
stopwords_spacy.remove('dias')
stopwords_spacy.remove('grandes')
stopwords_spacy.remove('general')

In [ ]:
import nltk

#Stop Words de nltk
from nltk.corpus import stopwords
stopwords_nltk = set(stopwords.words('spanish'))

# Quitamos de las stopwords la palabra 'no', que nos puede ser últil para predecir
stopwords_nltk.remove('no')

import re
from nltk.tokenize import RegexpTokenizer

In [ ]:
# Clase para personalizar las impresiones de consola
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Importar y concatenar los datasets

In [ ]:
# Importamos dataset de desarrollo
data_dev = pd.read_json("dataset_amazon/dataset_es_dev.json", lines = True)
print("- Cantidad de filas del set de " + color.YELLOW + "desarrollo" + color.END + ":", data_dev.shape[0])

# Importamos dataset de entrenamiento
data_train = pd.read_json("dataset_amazon/dataset_es_train.json", lines = True)
print("- Cantidad de filas del set de " + color.CYAN + "entrenamiento" + color.END + ":", data_train.shape[0])

# Importamos dataset de prueba
data_test = pd.read_json("dataset_amazon/dataset_es_test.json", lines = True)
print("- Cantidad de filas del set de " + color.GREEN + "prueba" + color.END + ":", data_test.shape[0])

In [ ]:
# Concatenamos todos
data = pd.concat([data_dev,data_train,data_test])
data = data.reset_index(drop=True)
data.tail()

In [ ]:
# Liberamos espacio de memoria
del(data_dev)
del(data_train)
del(data_test)
gc.collect()

### Filtrado de columnas
Nos quedamos con las columnas del dataset que son relevantes para predecir la valoración en general. Por eso se descartan las columnas de id de usuario y producto. La de categoría nos puede servir para relacionar palabras. La del idioma es redundante.

In [ ]:
df = data[['product_category','review_title','review_body','stars']]
df.head()

### Limpieza de datos

In [ ]:
#Constante de signos de puntuación
import string
puntua = string.punctuation + '¡¿...'
excluded_pos = ['SCONJ','CCONJ','NUM','PUNCT','PRON','DET','ADP','AUX','X']

In [ ]:
#Función para limpieza de datos con lemmatizer
def text_data_lemma(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = token.lemma_.strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

In [ ]:
from nltk.stem.snowball import SpanishStemmer # Permite stemmizar palabras en español
stemmer = SpanishStemmer()

#Función para limpieza de datos con stemmer
def text_data_stem(sentence):
    doc = nlp(sentence)
    
    clean_tokens = []
    for token in doc:
        if (token.pos_ not in excluded_pos and str(token) not in stopwords_spacy and len(token.text)>2): 
            temp = stemmer.stem(token.text).strip()
            clean_tokens.append(temp.lower())
    
    return clean_tokens

#### Lemmatización (40 min aprox)

In [ ]:
# Limpiamos todas las reviews con lemmatizer
reviews_lemma = []
for i in df.index:
    rev = text_data_lemma(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_lemma.append(" ".join(rev))
reviews_lemma[:5]

#### Stemmización (40 min aprox)

In [ ]:
# Limpiamos todas las reviews con stemmizer
reviews_stem = []
for i in df.index:
    rev = text_data_stem(df.review_title.iloc[i] + ' ' + df.review_body.iloc[i])
    reviews_stem.append(" ".join(rev))
reviews_stem[:5]

### Guardar nuevos datos en achivo json

In [ ]:
# Agregamos columna al dataset
df['revs_lemma'] = reviews_lemma
df['revs_stem'] = reviews_stem
df.head()

In [ ]:
# Guardamos dataset lemmatizado
df.to_json(path_or_buf='dataset_amazon_clean.json')